In [117]:
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MultiLabelBinarizer
import statsmodels.api as sm
pd.set_option('display.max_columns', 500)
plt.rcParams["figure.figsize"] = (30, 10)

In [118]:
data = pd.read_csv('df.csv')
data

,aso,brand,capacity,color,condition,features,first_owner,fuel_type,horse_power,mileage,model,no_accidents,number_of_doors,origin_country,price,price_currency,transmission,type,url,version,year
0,Tak,Rolls-Royce,6 749 cm3,Czarny,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Benzyna,599 KM,49 000 km,Cullinan,Tak,5,NaN,1616999,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-cull...,Black Badge,2020
1,NaN,Rolls-Royce,6 750 cm3,Niebieski,Używane,NaN,NaN,Benzyna,335 KM,48 000 km,Silver Spur,NaN,5,NaN,298800,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1991
2,Tak,Rolls-Royce,6 592 cm3,Szary,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,632 KM,1 800 km,Wraith,Tak,2,Włochy,1950000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,2019
3,NaN,Rolls-Royce,5 379 cm3,Biały,Używane,"ABS,Immobilizer,Alufelgi,Gniazdo SD,Podgrzewan...",Tak,Benzyna,326 KM,86 400 km,Silver Seraph,NaN,4,NaN,197000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
4,NaN,Rolls-Royce,4 000 cm3,Czarny,Używane,NaN,Tak,Benzyna,130 KM,50 000 km,Wraith,NaN,4,Wielka Brytania,25000,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,1952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/W0VBD6EC8KG37603...,NaN,NaN
93266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/WVGZZZ1TZHW11255...,NaN,NaN
93267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/WVGZZZ1TZHW11255...,NaN,NaN
93268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/TMBCE7NP8G708332...,NaN,NaN


# Co do zrobienia z danymi? 
- aso na 0-1 X 
- OH brand X
- capacity na int X
- OH color X
- condition 0-1 X
- OH features X 
- first_owner bool X 
- fuel_type OH X 
- horse_power na int X 
- mileage  git na int X
- no_accidents X
- number_of_doors X 
- origin country OH 
- price - Y 
- transmission - if automatic 
- year 

In [119]:
def map_aso(row): 
    if row == 'Tak': 
        return 1 
    else: 
        return 0 

def map_capacity(row): 
    split_list = row.split(' ')
    try:  
        if len(split_list) == 2:
            return int(''.join(split_list[:1]))
        else: 
            return int(''.join(split_list[:2]))
    except ValueError: 
        return row
        
def map_condition(row): 
    if row == 'Używane':
        return 0
    else: 
        return 1 

def map_transmission(row): 
    if row == 'Automatyczna': 
        return 1 
    else :
        return 0 
        

In [120]:
data = data.dropna(subset=['features', 'capacity', 'horse_power', 'mileage'])
data['features'] = data['features'].dropna().apply(lambda x: x.split(','))

lb = MultiLabelBinarizer()
features = lb.fit_transform(data['features'])
features = pd.DataFrame(features, columns=lb.classes_)

df = pd.concat([data.drop(columns='features').reset_index(drop=True), features], axis=1)
df['aso'] = df['aso'].apply(map_aso)
df = pd.concat([df.drop(columns='brand'), pd.get_dummies(df['brand'], drop_first=True)], axis=1)
df['capacity'] = df['capacity'].dropna().apply(map_capacity)
df['horse_power'] = df['horse_power'].dropna().apply(map_capacity)
df['mileage'] = df['mileage'].dropna().apply(map_capacity)
df = df.loc[df['fuel_type'].isin(['Benzyna', 'Diesel', 'Hybryda', 'Benzyna+LPG', 'Elektryczny'])]
df = pd.concat([df.drop(columns='fuel_type'), pd.get_dummies(df['fuel_type'], drop_first=True)], axis=1)
df['no_accidents'] = df['no_accidents'].apply(map_aso)
df['first_owner'] = df['first_owner'].apply(map_aso)
df['condition'] = df['condition'].apply(map_condition)
# df = pd.concat([df.drop(columns='color'), pd.get_dummies(df['color'], drop_first=True)], axis=1)
df['transmission'] = df['transmission'].apply(map_condition)
df = (df.rename(columns={'transmission': 'automatyczna', 'condition': 'new'})
        .drop(columns=['origin_country', 'model', 'type', 'url', 'version', 'price_currency', 'color'])
        .dropna()
        .astype(int))


/tmp/ipykernel_275626/2430665440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['features'] = data['features'].dropna().apply(lambda x: x.split(','))


In [121]:
df

,aso,capacity,new,first_owner,horse_power,mileage,no_accidents,number_of_doors,price,automatyczna,year,ABS,ASR (kontrola trakcji),Alarm,Alufelgi,Asystent parkowania,Asystent pasa ruchu,Bluetooth,CD,Centralny zamek,Czujnik deszczu,Czujnik martwego pola,Czujnik zmierzchu,Czujniki parkowania przednie,Czujniki parkowania tylne,Dach panoramiczny,ESP (stabilizacja toru jazdy),Elektrochromatyczne lusterka boczne,Elektrochromatyczne lusterko wsteczne,Elektryczne szyby przednie,Elektryczne szyby tylne,Elektrycznie ustawiane fotele,Elektrycznie ustawiane lusterka,Gniazdo AUX,Gniazdo SD,Gniazdo USB,HUD (wyświetlacz przezierny),Hak,Immobilizer,Isofix,Kamera cofania,Klimatyzacja automatyczna,Klimatyzacja czterostrefowa,Klimatyzacja dwustrefowa,Klimatyzacja manualna,Komputer pokładowy,Kurtyny powietrzne,MP3,Nawigacja GPS,Odtwarzacz DVD,Ogranicznik prędkości,Ogrzewanie postojowe,Podgrzewana przednia szyba,Podgrzewane lusterka boczne,Podgrzewane przednie siedzenia,Podgrzewane tylne siedzenia,Poduszka powietrzna chroniąca kolana,Poduszka powietrzna kierowcy,Poduszka powietrzna pasażera,Poduszki boczne przednie,Poduszki boczne tylne,Przyciemniane szyby,Radio fabryczne,Radio niefabryczne,Regulowane zawieszenie,Relingi dachowe,System Start-Stop,Szyberdach,Tapicerka skórzana,Tapicerka welurowa,Tempomat,Tempomat aktywny,Tuner TV,Wielofunkcyjna kierownica,Wspomaganie kierownicy,Zmieniarka CD,features,Łopatki zmiany biegów,Światła LED,Światła Xenonowe,Światła do jazdy dziennej,Światła przeciwmgielne,Audi,Bentley,Chevrolet,Citroën,Dacia,Ferrari,Fiat,Kia,Lamborghini,Land Rover,Lexus,MINI,Maserati,Mazda,McLaren,Mercedes-Benz,Mitsubishi,Porsche,Renault,Rolls-Royce,Saab,Seat,Suzuki,Toyota,Volvo,brand,Škoda,Benzyna+LPG,Diesel,Hybryda
0,1,6749,0,1,599,49000,1,5,1616999,1,2020,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,6592,0,0,632,1800,1,2,1950000,1,2019,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,1,1,0,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,5379,0,1,326,86400,0,4,197000,1,1999,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,6750,0,0,200,650000,0,4,59900,1,1975,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1,5379,0,0,326,58650,0,4,189900,1,1999,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71976,1,1969,0,1,190,175000,1,5,139000,1,2017,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
71977,1,1560,0,0,109,249000,1,5,14700,1,2008,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
71978,1,1969,0,1,320,110000,1,5,223860,1,2016,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

In [122]:
df.isna().sum(axis=0)

aso            0
capacity       0
new            0
first_owner    0
horse_power    0
              ..
brand          0
Škoda          0
Benzyna+LPG    0
Diesel         0
Hybryda        0
Length: 112, dtype: int64

In [123]:
df.dtypes.value_counts()

int64    112
dtype: int64

In [124]:
Y = df['price']
X = df.loc[:, df.columns != 'price']
lm = sm.OLS(Y, X).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     2154.
Date:                Sun, 16 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:26:42   Log-Likelihood:            -8.8955e+05
No. Observations:               71627   AIC:                         1.779e+06
Df Residuals:                   71518   BIC:                         1.780e+06
Df Model:                         108                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [125]:
X2 = df.drop(columns=lb.classes_).drop(columns='price')
Y = df['price']
lm = sm.OLS(Y, X2).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     5287.
Date:                Sun, 16 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:26:43   Log-Likelihood:            -8.9353e+05
No. Observations:               71627   AIC:                         1.787e+06
Df Residuals:                   71588   BIC:                         1.787e+06
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
aso              1486.9310    621.078     